# Download the data

In [13]:
%%bash
# wget --user=anonymous --password='yuma@indiana.edu' -r -nH --cut-dirs=4 --no-parent --reject "index.html*" http://garner.ucsd.edu/pub/timeseries/measures/ats/WesternNorthAmerica/

All paths as in properties.py but with changed UNIX style paths for the code to execute across platforms

In [1]:
import os 
V={}
WORK_DIR = '/media/hru/Data/_Active_Projects/PythonRDAHMM/test'
V['cron_path']=os.path.join(WORK_DIR, "RDAHMM","CRON_Download/")  
V['download_path']=os.path.join(WORK_DIR,"RDAHMM","Download/")  
V['script_path']=os.path.join(WORK_DIR,"PythonRDAHMM/") 
V['data_path']=os.path.join(WORK_DIR,"RDAHMM","Data/")
# temp_path is the temporary working directory for ingesting raw data
V['temp_path']=os.path.join(WORK_DIR,"RDAHMM","TEMP/")
V['model_path']=os.path.join(WORK_DIR,"RDAHMM","Model/")
V['eval_path']=os.path.join(WORK_DIR,"daily/")
V['train_epoch']="2013-12-31"
V['rdahmm_bin']=os.path.join(WORK_DIR,"RDAHMM", "rdahmm3","bin", "rdahmm")
V['rdahmm_model_parm']="-data <inputFile> -T <dataCount> -D <dimensionCount> -N 5 -output_type gauss -anneal -annealfactor 1.1 -betamin 0.1 -regularize -omega 0 0 1 1.0e-6 -ntries 10 -seed 1234"
V['rdahmm_eval_parm']="-data <proBaseName>.all.input -T <dataCount> -D <dimensionCount> -N 5 -output_type gauss -A <modelBaseName>.A -B <modelBaseName>.B -pi <modelBaseName>.pi -minvalfile <modelBaseName>.minval -maxvalfile <modelBaseName>.maxval -rangefile <modelBaseName>.range -eval"
V['dygraphsJs']=os.path.join(WORK_DIR,"PythonRDAHMM","dygraphsJsCreator.pearl")

def properties(key):
    return V[key]

# Read Reno data and add it to database

In [2]:
properties('cron_path')

'/media/hru/Data/_Active_Projects/PythonRDAHMM/test/RDAHMM/CRON_Download/'

In [16]:
# %run scripps_ingest_single.py /media/hru/Data/_Active_Projects/PythonRDAHMM/test/RDAHMM/Download/WesternNorthAmerica/WNAM_Clean_DetrendNeuTimeSeries_comb_20160914.tar

Running this file generated two folders in the RAHMM folder-
1. <a href="/media/hru/Data/_Active_Projects/PythonRDAHMM/test/RDAHMM/TEMPWNAM_Clean_DetrendNeuTimeSeries_comb
" >TEMPWNAM_Clean_DetrendNeuTimeSeries_comb </a>
2. <a href="/media/hru/Data/_Active_Projects/PythonRDAHMM/test/RDAHMM/DataWNAM_Clean_DetrendNeuTimeSeries_comb
"> DataWNAM_Clean_DetrendNeuTimeSeries_comb: </a> Contains the SQLite database for each station


gf9 already has a number of data sets and models trained on each of them. I am considering only the <b><i>rawNeuTimeSeries.MEASURES_Combination</i></b>. But these database dont have the raw files.

In [2]:
import os, sys, string, re
import sqlite3 as db
from datetime import date
from datetime import timedelta
# from properties import properties

In [4]:
argv1 = "/media/hru/Data/_Active_Projects/PythonRDAHMM/test/RDAHMM/Download/WesternNorthAmerica/WNAM_Clean_DetrendNeuTimeSeries_comb_20160914.tar"
[scripps_path, tarfile] = os.path.split(argv1)
scripps_path += "/"

In [5]:
data_path = properties('data_path')
temp_path = properties('temp_path')

In [30]:
datadir = data_path + tarfile[:tarfile.rfind("_")] + "/"
#dbfile = datadir + tarfile[:-4] + ".sqlite"
# get rid of timestamp from db file name
dbfile = datadir + tarfile[:-13] + ".sqlite"
workdir = temp_path + tarfile[:tarfile.rfind("_")] + "/"

In [31]:
if not os.path.exists(datadir):
    cmd = "mkdir -p " + datadir
    os.system(cmd) 
if not os.path.exists(workdir):
    cmd = "mkdir -p " + workdir
    os.system(cmd) 

In [32]:
#if the same db file exists, drop it
if os.path.isfile(dbfile):
    print "deleting old database " + dbfile
    os.remove(dbfile)

In [33]:
# # creating/connecting the database 
# conn = db.connect(dbfile)
# # creating a Cursor
# cur = conn.cursor()
# # creating tables
# sql ="""CREATE TABLE GPSTimeSeries (
#       StationID CHAR(4), 
#       North Num,
#       East Num,
#       Up  Num,
#       Nsig Num,
#       Esig Num, 
#       Usig Num,
#       Timestamp TEXT,
#       UNIQUE (StationID, Timestamp))"""
# cur.execute(sql)
# sql ="""CREATE TABLE ReferencePositions (
#       StationID CHAR(4), 
#       Latitude Num,
#       Longitude Num, 
#       Height Num, 
#       UNIQUE (StationID))"""
# cur.execute(sql)
# conn.commit()

In [34]:
# clear working directory
cmd = "rm -f " + workdir + "*"
os.system(cmd)

0

In [35]:
print "Processing ", tarfile, "..." 
# unpack data 
# cmd = "tar xvf " + scripps_path + tarfile + " -C " + workdir
# os.system(cmd)
dirlist = os.listdir(workdir)
dirlist.sort()

Processing  WNAM_Clean_DetrendNeuTimeSeries_comb_20160914.tar ...


In [36]:
# i kept only 1 zip files(data for 1 station) to improve access speed
workdir
dirlist = os.listdir(workdir)
dirlist.sort()

In [37]:
for datafile in dirlist:
    if datafile[-2:] == ".Z":
        if os.stat(workdir+datafile).st_size == 0: # When .Z file is empty
            continue
        print datafile
        cmd = "unzip " + workdir + datafile + " -d " + workdir
        os.system(cmd)
        datafile = datafile[:-2]
        stationID = datafile[:4]
        
        print datafile
        print stationID
#         station_dbfile = datadir + stationID + ".sqlite"
#         if os.path.isfile(station_dbfile):
#             print "deleting old station database " + station_dbfile
#             os.remove(station_dbfile)
#         station_conn = db.connect(station_dbfile)
#         station_cur = station_conn.cursor()
#         station_sql ="""CREATE TABLE StationGPSTimeSeries (
#                       North Num,
#                       East Num,
#                       Up  Num,
#                       Nsig Num,
#                       Esig Num, 
#                       Usig Num,
#                       Timestamp TEXT,
#                       Interploated INT Default 0,
#                       UNIQUE(Timestamp))"""
#         station_cur.execute(station_sql)
#         station_conn.commit()

    with open(workdir + datafile, 'r') as f:
        data = f.readlines()
        last_line = ""
        for line in data:
            if "Reference position" in line:
                refs = map(float, re.findall("(-?[0-9.]*[0-9]+)", line))
                lat = refs[0] + refs[1]/60.0 + refs[2]/3600.0
                long = -1.0 * (refs[3] + refs[4]/60.0 + refs[5]/3600.0)
                height = refs[6]
#                 sql = "INSERT INTO ReferencePositions (StationID, Latitude, Longitude, Height) "
#                 sql += " VALUES ('%s', '%s', '%s', '%s')" % (stationID, lat, long, height)
#                 cur.execute(sql)
#             if not "#" in line:
#                 record = string.split(line)
# 		if len(record) < 9: # When missing white spaces between columns
# 		    tmpstr = ' '.join(record[3:])
#                     record[3:9] = re.findall(r"[+-]?\d+\.\d\d", tmpstr)
#                 [year, days] = map(int, record[1:3])
# 	        # days is counted starting from 0
#                 timestamp = date.fromordinal(date(year,1,1).toordinal()+days)
#                 [north, east, up, nsig, esig, usig] = record[3:9]
#                 sql = "INSERT INTO GPSTimeSeries (StationID, North, East, Up, Nsig, Esig, Usig, Timestamp) "
#                 sql += " VALUES ('%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s')" % (stationID, north, east, up, nsig, esig, usig, timestamp)
#                 cur.execute(sql)
                
#                 if last_line == "":
#                     last_line = line
#                 else:
#                     last_record = string.split(last_line)
# 		    if len(last_record) < 9:
# 		        tmpstr = ' '.join(last_record[3:])
#                         last_record[3:9] = re.findall(r"[+-]?\d+\.\d\d", tmpstr)
#                     [year, days] = map(int, last_record[1:3])
# 	            # days is counted starting from 0
#                     last_timestamp = date.fromordinal(date(year,1,1).toordinal()+days)
#                     [lnorth, least, lup, lnsig, lesig, lusig] = last_record[3:9]
#                     # if missing days from last to current, fill with last
#                     for i in range(1, (timestamp - last_timestamp).days):
#                         ts = last_timestamp + timedelta(days=i)
#                         interploated = 1
#                         station_sql = "INSERT INTO StationGPSTimeSeries (North, East, Up, Nsig, Esig, Usig, Timestamp, Interploated) "
#                         station_sql += " VALUES ('%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s')" % (lnorth, least, lup, lnsig, lesig, lusig, ts, interploated)
#                         station_cur.execute(station_sql)
#                     last_line = line

#                 station_sql = "INSERT INTO StationGPSTimeSeries (North, East, Up, Nsig, Esig, Usig, Timestamp) "
#                 station_sql += " VALUES ('%s', '%s', '%s', '%s', '%s', '%s', '%s')" % (north, east, up, nsig, esig, usig, timestamp)
#                 station_cur.execute(station_sql)

# Read JPL data and add it to database

## Download JPL data

Source: ftp://sideshow.jpl.nasa.gov/pub/usrs/mbh/point/

In [3]:
from ftplib import FTP

In [4]:
data_path = properties('data_path')
datadir = data_path + "JPL"
datadir

'/media/hru/Data/_Active_Projects/PythonRDAHMM/test\\RDAHMM\\Data/JPL'

In [5]:
def make_sure_path_exists(path): 
    '''
    Function to  make a new directory structure if it doesnt exists.
    Created by following: 
    # http://stackoverflow.com/questions/273192/how-to-check-if-a-directory-exists-and-create-it-if-necessary
    Input: Directory path
    Output: Creates directories if they dont exist
    '''
    if not os.path.exists(path):
        os.makedirs(path)

make_sure_path_exists(datadir)

In [26]:
def connect(server='sideshow.jpl.nasa.gov'):
    '''
    Function to establish ftp connection to server
    
    Input: url to server. By default it is the JPL server url
    '''

    ftp = FTP(server)     # connect to host, default port
    ftp.login()                     # user anonymous, passwd anonymous@
    return ftp

In [27]:
# http://stackoverflow.com/questions/3451817/python-ftplib-timing-out
# ftp.set_pasv(False)
# http://stackoverflow.com/questions/5230966/python-ftp-download-all-files-in-directory

In [28]:
ftp = connect()
ftp.cwd('pub/usrs/mbh/point') # Change working directory to time series folder

'250 CWD command successful'

In [29]:
filenames = ftp.nlst() # get filenames within the directory
print filenames

['NOVM.series', 'NPIT.series', 'NPLD.series', 'NPLY.series', 'NPOC.series', 'NPRG.series', 'NPRI.series', 'NRC1.series', 'NRC2.series', 'NRIF.series', 'NRIL.series', 'NRL1.series', 'NRMD.series', 'NRWY.series', 'NSSP.series', 'NSSS.series', 'NSTG.series', 'NTKA.series', 'NTUS.series', 'NUNE.series', 'NURK.series', 'NVSK.series', 'NWCC.series', 'NYA1.series', 'NYAL.series', 'NZRT.series', 'OAES.series', 'OAT2.series', 'OBE2.series', 'OBEC.series', 'OBER.series', 'OBET.series', 'OCSD.series', 'ODS5.series', 'OEOC.series', 'OFW2.series', 'OFWY.series', 'OGHS.series', 'OHI2.series', 'OHIG.series', 'OHLN.series', 'OKEE.series', 'OKIT.series', 'OKOM.series', 'OMH1.series', 'ONSA.series', 'OPBL.series', 'OPCL.series', 'OPCP.series', 'OPCX.series', 'OPMT.series', 'OPRD.series', 'ORES.series', 'ORID.series', 'ORMT.series', 'ORO_.series', 'ORVB.series', 'OS0G.series', 'OSJE.series', 'OSN1.series', 'OSN2.series', 'OUS2.series', 'OUSD.series', 'OVLS.series', 'OXMT.series', 'OXYC.series', 'OZST.ser

In [30]:
def complete_data_downloaded(web_list, datadir):
    '''
    Function to check if all the files are downloaded. 
    It was implemented as the connection to the Nasa server
    gets timeout and drops many times.
    
    Input: 
        web_list: list containing the files that need to downloaded
        datadir: the directory where files are being downloaded
        
    Output: Boolean indicating if download is complete
    '''
    
#     http://stackoverflow.com/questions/11697709/comparing-two-lists-in-python
    file_list = os.listdir(datadir)
    common_files = list(set(file_list) & set(filenames))    
    if  len(common_files)== len(web_list):
        return True
    else:
        print("Missing " + str(len(web_list) - len(common_files)) + " files")
        return False
    
complete_data_downloaded(filenames, datadir)

True

In [31]:
connect_count = 0
while not complete_data_downloaded(filenames, datadir):
    try:
        for filename in filenames:
            local_filename = os.path.join(datadir, filename)
            if not os.path.exists(local_filename):
                print("Writing to: "+ local_filename)
                file = open(local_filename, 'wb')
                ftp.retrbinary('RETR '+ filename, file.write)

                file.close() 
            else:
                print(local_filename+" exists")
    except:
        ftp = connect()
        ftp.cwd('pub/usrs/mbh/point') 
        connect_count +=1

ftp.quit()

'221 Goodbye.'

## Understand structure of JPL dataset and clean if needed

Source: http://sideshow.jpl.nasa.gov/post/tables/GPS_Time_Series.pdf

Time Series and Residual Format  
Column 1: Decimal_YR  
Columns 2-4: East(m) North(m) Vert(m)  
Columns 5-7: E_sig(m) N_sig(m) V_sig(m)  
Columns 8-10: E_N_cor, E_V_cor, N_V_cor  
Column 11: Time in Seconds past J2000  
Columns 12-17: Time in YEAR MM DD HR MN S

In [9]:
path_raw_data_jpl = os.path.join(properties('download_path'), 'JPL')
path_raw_data_jpl

'/media/hru/Data/_Active_Projects/PythonRDAHMM/test\\RDAHMM\\Download/JPL'

In [10]:
import pandas as pd

In [11]:
count = 0;
empyt_data_stations = []

for series in os.listdir(path_raw_data_jpl):
#     df = pd.read_table(os.path.join(path_raw_data_jpl,series), \
#                        sep='         ', header=None, engine='python')
    if series != 'README':
        with open(os.path.join(path_raw_data_jpl,series)) as stations_file:
            data = stations_file.readlines()

            for idx, row in enumerate(data):
                data[idx] = string.split(data[idx])

            df = pd.DataFrame(data)

            if df.isnull().sum().sum() or len(df.columns) != 17:
                empyt_data_stations.append(series)

            count+=1
#     if count ==0:
#         break

        

WindowsError: [Error 3] The system cannot find the path specified: '/media/hru/Data/_Active_Projects/PythonRDAHMM/test\\RDAHMM\\Download/JPL/*.*'

In [ ]:
print "No of files read:", count
print "No of stations with null data: "
for e in empyt_data_stations:
    print e

<b> NO NEED TO CLEAN DATA</b>

17

## Add JPL to database

# Train model
But as training model was available on gf9 I used that, since training the model takes 2 weeks.

In [4]:
argv1 = "WNAM_Clean_DetrendNeuTimeSeries_comb"
dataset = argv1

In [5]:
data_path = properties('data_path') + "/" + dataset + "/"
model_path = properties('model_path') + "/" + dataset + "/"
train_epoch = properties('train_epoch')
rdahmm_bin = properties('rdahmm_bin')

# Evaluate model
the output is stored in daily_project_stationID_eval-date.Q file in the same directory as the training model

In [15]:
import sqlite3
sqlite_file = '/home/hru/Desktop/my_db.sqlite'
conn = sqlite3.connect(sqlite_file)
c = conn.cursor()

In [17]:
station_sql = """CREATE TABLE StationGPSTimeSeries (
                       North Num,
                       East Num,
                       Up  Num,
                       Nsig Num,
                       Esig Num, 
                       Usig Num,
                       Timestamp TEXT,
                       Interploated INT Default 0,
                       UNIQUE(Timestamp))"""
conn.execute(station_sql)
conn.commit()

OperationalError: table StationGPSTimeSeries already exists

In [24]:
import datetime
with open('/media/hru/Data/_Active_Projects/PythonRDAHMM/test/RDAHMM/Download/JPL/7ODM.series', 'r') as f:
        data = f.readlines()
        station_sql =""
        last_line = ""
        for line in data:
            last_record = string.split(line)
            #lsts = last_record[1]
            least = float(last_record[1])*1000.0
            lnorth = float(last_record[2])*1000.0
            lup = float(last_record[3])*1000.0
            lesig = float(last_record[4])*1000.0
            lnsig = float(last_record[5])*1000.0
            lusig = float(last_record[6])*1000.0
            last_timestamp = datetime.datetime(int(last_record[11]),int(last_record[12]),int(last_record[13])
            # if missing days from last to current, fill with last
            station_sql = "INSERT INTO StationGPSTimeSeries (North, East, Up, Nsig, Esig, Usig, Timestamp) "
            station_sql += " VALUES ('%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s')" % (lnorth, least, lup, lnsig, lesig, lusig, last_timestamp)
            conn.execute(station_sql))
        conn.commit()

SyntaxError: invalid syntax (<ipython-input-24-18c6534bad09>, line 17)

In [ ]:
if last_line == "":
    last_line = line
else:
last_record = string.split(last_line)
#lsts = last_record[1]
least = float(last_record[1])*1000.0
lnorth = float(last_record[2])*1000.0
lup = float(last_record[3])*1000.0
lesig = float(last_record[4])*1000.0
lnsig = float(last_record[5])*1000.0
lusig = float(last_record[6])*1000.0
last_timestamp = datetime.datetime(last_record[11],last_record[12],last_record[13])
    # if missing days from last to current, fill with last
for i in range(1, (timestamp - last_timestamp).days):
    ts = last_timestamp + timedelta(days=i)
    interploated = 1
    station_sql = "INSERT INTO StationGPSTimeSeries (North, East, Up, Nsig, Esig, Usig, Timestamp) "
    station_sql += " VALUES ('%s', '%s', '%s', '%s', '%s', '%s', '%s', '%s')" % (lnorth, least, lup, lnsig, lesig, lusig, ts)
    station_cur.execute(station_sql)
last_line = line